In [42]:
import sqlite3 
import pandas as pd 
import requests
import json  
from datetime import datetime
import secrets
import telegram #pip install python-telegram-bot --upgrade
import time
import math

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 500) 
pd.set_option('display.max_rows', 500) 

def connect_create_table(name_db='db.db'):
    conn = sqlite3.connect(name_db)  # You can create a new database by changing the name within the quotes
    c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

    # Create table - CLIENTS
    c.execute('''CREATE TABLE if not exists POSITIONS
                 ([id] INTEGER PRIMARY KEY,
                 [date] DATETIME,
                 [encryptedUid] TEXT,
                 [nick] TEXT,
                 [symbol] TEXT,
                 [status] TEXT,
                 [entryPrice] FLOAT,
                 [markPrice] FLOAT,
                 [pnl] FLOAT,
                 [roe] FLOAT,
                 [amount] FLOAT,
                 [amount_usd] FLOAT,
                 [updateTimeStamp] TEXT,
                 [request_id] INT
                 )''')
    conn.commit()

connect_create_table()

In [43]:
def human_format(num):
    num = float(num)
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [44]:
def insert_in_table(trader_info, request_id, vector, status):
    encryptedUid = trader_info[0]
    nick         = trader_info[1]
    
    try:
        sqliteConnection = sqlite3.connect('db.db')
        cursor = sqliteConnection.cursor()
        
        symbol      = vector[0]
        entryPrice  = vector[1]
        markPrice   = vector[2]
        pnl         = vector[3]
        row         = vector[4]
        try: amount = float(vector[5])
        except: amount = 0
        amount_usd  = round(markPrice * amount,0)
        updateTimestamp = time.time()
        date        = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Need to add more info of student (such as the txns)
        sqlite_insert_with_param = """INSERT INTO POSITIONS 
                          (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, roe, amount, amount_usd, updateTimeStamp, request_id) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?,?);"""

        data_tuple = (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, row, amount, amount_usd, updateTimestamp,request_id) 
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()

In [45]:
def do_query(db="db.db", query="SELECT * FROM POSITIONS"):    
    con = sqlite3.connect(db)
    df = pd.read_sql_query(query, con)
    
    return df

In [46]:
def get_positions(trader):
    endpoint = "https://www.binance.com/bapi/futures/v1/public/future/leaderboard/getOtherPosition"
    params   =  {"encryptedUid":trader,  "tradeType":"PERPETUAL"}
    headers  =  {"content-type":"application/json;charset=UTF-8"}
    response = requests.post(endpoint, json=params, headers=headers) #Add proxies = proxyDict # Attention to JSON = PARAMS
    result   = response.json()
    
    return result

In [70]:
def send_message(text, chat_id = "-1001455341077"):
    my_bot  = telegram.Bot("814569733:AAGmMk_P4p8EdLkj-fj2fVziyByxHIcJpm0")
    my_bot.send_message(chat_id=chat_id, text=text, parse_mode = "Markdown")

In [71]:
def send_message_discord(text):
    #Webhook of my channel. Click on edit channel --> Webhooks --> Creates webhook
    webhook = "https://discord.com/api/webhooks/892399337731133500/_IE9zchjLp3lRrImBt8sp_YgrI8A9Kan1Vc6wv7eY7vvkVNEAGF-YkVkiUF4pKxT5J3R"
    data    = {"content": text}
    response = requests.post(webhook, json=data)

In [92]:
def create_message(trader, row, trader_max_size):

    if row["difference"] > 0: trade_type = "BUY"
    else: trade_type = "SELL" 
        
    status_new       = str(row["status_new"])
    status_old       = str(row["status_old"])
    if status_new =="OPEN" and status_old =="OPEN": action = "Modify existent trade"
    if status_new =="OPEN" and status_old !="OPEN": action = "Open a new trade"
    if status_new !="OPEN" and status_old =="OPEN": action = "Close existent trade"
        
    new_position     = round(row["amount_new"],4)
    old_position     = round(row["amount_old"],4)
    price            = str(round(row["markPrice_old"],4))
    difference       = str(abs(row["difference"]))
    symbol           = str(row["symbol"])
    date             = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    amount_usd_new       = row["amount_usd_new"]
    amount_usd_old       = row["amount_usd_old"]
    new_position_percent = str(round(100* amount_usd_new / trader_max_size,3))
    old_position_percent     = str(round(100* amount_usd_old / trader_max_size,3))
    delta_percent    = str(round(abs(float(new_position_percent) - float(old_position_percent)),3))
    
    message =  "💰" + trader + " " + date + "\n*" + action + "*\nPosition change from " + str(old_position) + symbol + " to " + str(new_position) + symbol + ".\n" + trade_type + " " + difference + symbol + " at " + price + "$" + "\nTrade went from " + old_position_percent + "% to "+ new_position_percent +"% (" + trade_type + " " + delta_percent + "%)"
    return message 

In [93]:
def message_logic(row, trader_dict, trader_max_size):
    elite_traders = ["TraderT","ADnefina","Nothingness","BBOS"]
    
    if row["difference"] != 0:   
        if   trader_dict in elite_traders:  chat_id = -542063577
        elif trader_dict == "ClickHereNow": chat_id = -591308509
        else:   chat_id = -530891051    
        message = create_message(trader_dict, row, trader_max_size)

        try: 
            send_message(message, chat_id)
            print(message)
        except: print("Too many messages")
        if trader_dict in elite_traders : send_message_discord(message)
        time.sleep(2)

In [94]:
def get_positions_db(two_ids):
    # Get last update from trader.             
    this_id, last_id = two_ids["request_id"][0], two_ids["request_id"][1]

    # Get all the positions with that request_id
    new_positions   = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{this_id}'")
    old_positions   = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{last_id}'")

    # Compare the previous position with the new one, pair by pair
    both               = old_positions.merge(new_positions, "outer", on ="symbol", suffixes=('_old', '_new')).fillna(0) #Filling nas (if previous or new position is na, is because is not present, hence 0)
    both["difference"] = round(both["amount_new"] - both["amount_old"],4)
    print(both[["symbol","amount_old","amount_new","difference", "amount_usd_new"]])
    
    return both

In [95]:
def get_trader_max_size(trader,tol = 0.20):
    start = time.time()
    trades        = do_query(query=f"select amount_usd, request_id from positions where encryptedUid = '{trader}'")
    trades["amount_usd"]    = abs(trades["amount_usd"])
    sum_positions = trades.groupby(['request_id'])["amount_usd"].sum()
    position_max  = sum_positions.max()
    trader_max_size = position_max*(1+tol)
    print("time for getting trader max size:", round( time.time() - start,5))
    return trader_max_size

In [ ]:
names    = {"CCF3E0CB0AAD54D9D6B4CEC5E3E741D2":"TraderT", 
            "49A7275656A7ABF56830126ACC619FEB":"ADnefina", 
            "A532C4316C00206168F795EDFBB3E164":"ClickHereNow", 
            "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard",
            "FB23E1A8B7E2944FAAEC6219BBDF8243":"BananaMan1", 
            "F318AE1B1BB3AEF4EDBD36E5AE250CD1":"BigPiePlayer", 
            "2F9D01E80429F97670415A5A5DDD9405":"CryptoPinoy",
            "95ACEF9493428B28CCADDA38A2127859":"ChurchChalkUnder",
            "382066A3597798ED9F69FECBCABAA00D":"911korn",
            "15A34DAA83C4B2D7F3A4BBAB4D2FECCA":"BakeAlsoShare",
            "3E478F9133714216B608E95B4ACF1FA3":"SABSAB",
            "B936AEC059917C5A730D45922A9F4334":"GeboyMujaer",
            "8D27A8FA0C0A726CF01A7D11E0095577":"Nothingness",
            "C6EC5B8B2D08F5748A1D8D3D849023A3":"GetSwifty",
            "191792DA7B3EEB10DAF1D51B6926F2E1":"Mikkiko",
           "7BFD2EBA42119D988CAA64F0BA6C64C2":"BBOS"}

#names = {"A532C4316C00206168F795EDFBB3E164":"ClickHereNow"}

traders  = list(names.keys())
keys     = ["symbol", "entryPrice", "markPrice", "pnl", "roe", "amount"]

while True:
    start_trader = time.time()
    for trader in traders:
        print(" ")
        print("______________________", names[trader], "______________________")
        trader_info = [trader, names[trader]]    # Write the name of the trader
        trader_max_size = get_trader_max_size(trader)
        
        # Download data
        try:
            position_array   = get_positions(trader)     # Get the current position of the trader
            positions        = position_array["data"]["otherPositionRetList"]
            count = do_query(query="SELECT COUNT(*) as count FROM positions")["count"][0]
            if count > 0: request_id  = int(do_query(query="select MAX(id) as id from positions")["id"][0])
            else:         request_id = 0

        except:
            print(f"Couldn't download position for {names[trader]}")
            continue #This is useful not to mix stuff between positions and no positions        
        
        # Insert in the database what we downloaded
        if not positions:
            print(f"No positions for {names[trader]}, but still save.")
            empty    = [0,0,0,0,0,0] # Need to get the timestamp right!
            insert_in_table(trader_info, request_id, empty, "CLOSE")

        if positions:
            for i in range(len(positions)):
                row = pd.DataFrame(positions).loc[i]
                vector = [row["symbol"], row["entryPrice"], row["markPrice"], row["pnl"], row["roe"], row["amount"]]
                status = "CLOSED" if row["amount"] == 0 else "OPEN"   # bug, this displays 0
                insert_in_table(trader_info, request_id, vector, status)
        
        # Get last updates
        two_ids = do_query(query=f"select distinct(request_id) from POSITIONS where encryptedUid = '{trader}' order by request_id desc limit 2")
        
        if len(two_ids)>=2: # if the trader is in the database, check the delta
            both = get_positions_db(two_ids)

            for i in range(len(both)):
                row = both.loc[i]
                message_logic(row, names[trader], trader_max_size)
        
            time.sleep(2)
        
    print("####################################################################")
    print("Time for one cycle of traders:", round( time.time() - start_trader,2))
    print("####################################################################")

 
______________________ TraderT ______________________
time for getting trader max size: 0.00504
     symbol  amount_old  amount_new  difference  amount_usd_new
0   ETHUSDT     -584.01     -584.01        0.00     -2031192.00
1   SOLUSDT   -11899.00   -11899.00        0.00     -2091023.00
2   BTCUSDT      -90.52      -90.52        0.00     -4424878.00
3   FTMUSDT  -329417.00  -329417.00        0.00      -499026.00
4   DOTUSDT   -29072.50   -29072.50        0.00      -942879.00
5  DYDXUSDT    22806.20    22806.20        0.00       512661.00
 
______________________ ADnefina ______________________
time for getting trader max size: 0.00401
    symbol  amount_old  amount_new  difference  amount_usd_new
0  BTCUSDT      166.24      166.24        0.00      8126598.00
 
______________________ ClickHereNow ______________________
time for getting trader max size: 0.00656
          symbol  amount_old  amount_new  difference  amount_usd_new
0        BTCBUSD       24.66       24.66        0.00     

In [ ]:
do_query(query="select MAX(id) as id from positions")["id"][0]

In [ ]:
do_query(query=f"select distinct(request_id) from POSITIONS where encryptedUid = '{trader}' order by request_id desc limit 2")

In [ ]:
do_query(query="select*from positions where nick='ClickHereNow'")

    symbol  amount_old  amount_new  difference  amount_usd_new
0  BTCUSDT      163.19           0     -163.19               0


,id_old,date_old,encryptedUid_old,nick_old,symbol,status_old,entryPrice_old,markPrice_old,pnl_old,roe_old,amount_old,amount_usd_old,updateTimeStamp_old,request_id_old,id_new,date_new,encryptedUid_new,nick_new,status_new,entryPrice_new,markPrice_new,pnl_new,roe_new,amount_new,amount_usd_new,updateTimeStamp_new,request_id_new,difference
0,3,2021-10-02 01:21:32,49A7275656A7ABF56830126ACC619FEB,ADnefina,BTCUSDT,OPEN,47696.26,47992.77,48387.90,0.06,163.19,7832084.00,1633130492.74901,2,0,0,0,0,0,0,0,0,0,0,0,0,0,-163.19
